# Lab4

## Download NDAWN Data

In [60]:
import pandas as pd
import requests
from datetime import date,timedelta
from io import StringIO

class ndawn_request:

    def __init__(self, startDate='YYYY-MM-DD', endDate='YYYY-MM-DD', ontology = None, location = None, save = False):

        self.start = startDate

        self.end = endDate

        # List of ontology terms, and their URL codes to build request URL
        self.ontology = {
            'Air Temperature': ['variable=hdt', 'variable=hdt9'],
            'Relative Humidity': ['variable=hdrh', 'variable=hdrh9'],
            'Soil Temperature': ['variable=hdbst', 'variable=hdtst'],
            'Wind Speed': ['variable=hdws', 'variable=hdmxws', 'variable=hdws10', 'variable=hdmxws10'],
            'Wind Direction': ['variable=hdwd', 'variable=hdsdwd', '&variable=hdwd10', 'variable=hdsdwd10'],
            'Solar Radiation': ['variable=hdsr'],
            'Rainfall': ['variable=hdr'],
            'Air Pressure': ['variable=hdbp'],
            'Dew Point': ['variable=hddp'],
            'Wind Chill': ['variable=hdwc']
        }
        # Concatenate the ontology keys into a list for exception printout later
        ontologiesErrorMessage = '\n'.join(list(self.ontology.keys()))

        # List of stations, and URL codes to build request URL
        self.stations = {
            'Ada': 78,
            'Becker': 118,
            'Campbell': 87,
            'Clarissa': 124,
            'Eldred': 2,
            'Fox': 93,
            'Greenbush': 70,
            'Hubbard': 119,
            'Humboldt': 4,
            'Kennedy': 82,
            'Little Falls': 120,
            'Mavie': 71,
            'Ottertail': 103,
            'Parkers Prairie': 116,
            'Perham': 114,
            'Perley': 3,
            'Pine Point': 115,
            'Rice': 121,
            'Roseau': 61,
            'Sabin': 60,
            'Staples': 122,
            'Stephen': 5,
            'Ulen': 91,
            'Wadena': 117,
            'Warren': 6,
            'Waukon': 92,
            'Westport': 123,
            'Williams': 95
        }
        # Concatenate station names into a list for exception printout later
        stationsErrorMessage = '\n'.join(list(self.stations.keys()))

        self.save = save

        # This checks the start and end dates supplied to make sure they are valid
        # Start by converting dates into iso format
        startDateCheck = date.fromisoformat(startDate)
        endDateCheck = date.fromisoformat(endDate)
        # If start date is after end date, raise exception
        if startDateCheck > endDateCheck:
            raise Exception('End date cannot be before start date')
        
        # Create empty list to hold URL codes for ontology terms
        self.activeMeasures = []
        # If user supplies ontology terms
        if ontology is not None:
            for item in ontology:
                # If user-supplied term is not in the dictionary, raise exception
                if item not in self.ontology.keys():
                    raise Exception('Ontology term [' + str(item) + '] not recognized. Available ontology terms include: ' + '\n' + ontologiesErrorMessage)
                # Otherwise, append URL codes for ontology terms into the list of measurements to be requested
                else:
                    for code in self.ontology[item]:
                        self.activeMeasures.append(code)  
        # If user does not supply ontology terms, add all URL codes in dictionary to the list of measurements to be requested    
        else:
            for key in self.ontology:
                for code in self.ontology[key]:
                    self.activeMeasures.append(code)

        # Create empty list to hold URL codes for stations
        self.activeStations = []
        # If user supplies station names
        if location is not None:
            for name in location:
                # If user-supplied name is not in the dictionary, raise exception
                if name not in self.stations.keys():
                    raise Exception('Station [' + str(name) + '] not recognized. Available stations include: ' + '\n' + stationsErrorMessage)
                # Otherwise, append URL codes for stations into the list of stations to be requested
                else:
                    self.activeStations.append('station=' + str(self.stations[name]))
        # If user does not supply station names, add all station URL codes in dictionary to the list of stations to be requested
        else:    
            for key in self.stations:
                self.activeStations.append('station=' + str(self.stations[key]))

    def get_data(self):
        
        # Construct API call for the request
        baseURL = 'https://ndawn.ndsu.nodak.edu/table.csv?'
        stations = '&'.join(self.activeStations)
        measurements = '&'.join(self.activeMeasures)
        options = '&ttype=hourly&quick_pick=&begin_date=' + self.start + '&end_date=' + self.end
        finalURL = str(baseURL + stations + '&' + measurements + options)
        
        # Request page
        page = requests.get(finalURL)
        # If status code not 200, raise exception
        if page.status_code != 200:
            raise Exception('URL request status not 200. Status code = ' + page.status_code)

        print('Request successful')

        # Convert csv data to string
        content = str(page.content)
        # Remove large, unnecessary header
        trimContent = content[content.find('Station'):len(content)]
        # Replace newline/return with string literal newline
        formatContent = trimContent.replace('\\r\\n', '\n')
        # Convert content to file object
        contentFile = StringIO(formatContent)

        # Read content into pandas dataframe. Second header row contains units
        ndawnData = pd.read_csv(contentFile, header = [0, 1])
        
        # Concatenate headers to include units
        # Assign column list to object
        columnHeaders = list(ndawnData.columns)
        # List of new headers
        newHeaderList = []
        # Iterate through column names
        for number in range(0, len(columnHeaders)):
            # If no unit, keep header unchanged, pass into new list
            if 'Unnamed' in columnHeaders[number][1]:
                newHeaderList.append(columnHeaders[number][0])
            # If unit exists, concatenate header and unit, pass into new list
            else:
                newHeader = columnHeaders[number][0] + ' (' + columnHeaders[number][1] + ') '
                newHeaderList.append(newHeader)
        # Assign new column names
        ndawnData.columns = newHeaderList

        # Create single column for datetime
        ndawnData['Date'] = pd.to_datetime(ndawnData[['Year', 'Month', 'Day']])
        
        # Save to csv if save option selected
        if self.save:
            ndawnData.to_csv('ndawnData.csv', index=False)

        return ndawnData
    
"""
# Example syntax:
exampleRequest = ndawn_request(startDate='2020-06-23', endDate='2020-06-28', ontology=['Air Pressure', 'Relative Humidity', 'Soil Temperature', 'Wind Direction', 'Wind Speed'], location=['Mavie', 'Ottertail', 'Perham', 'Perley'])
ndawnDF = exampleRequest.get_data()
"""


"\n# Example syntax:\nexampleRequest = ndawn_request(startDate='2020-06-23', endDate='2020-06-28', ontology=['Air Pressure', 'Relative Humidity', 'Soil Temperature', 'Wind Direction', 'Wind Speed'], location=['Mavie', 'Ottertail', 'Perham', 'Perley'])\nndawnDF = exampleRequest.get_data()\n"

In [94]:
# Get the date of today and the date of 30 days ago
import datetime
today = datetime.date.today()
start = today - datetime.timedelta(days=29)
startdate=start.strftime("%Y-%m-%d")
end=datetime.date.today()
enddate = end.strftime("%Y-%m-%d")

In [120]:
#request the data from ndawn
exampleRequest = ndawn_request(startDate=startdate, endDate=enddate, ontology=['Air Temperature','Air Pressure', 'Relative Humidity', 'Soil Temperature', 'Wind Direction', 'Wind Speed'], location=['Ada','Becker','Campbell','Clarissa','Eldred','Fox','Greenbush','Hubbard','Humboldt','Kennedy','Little Falls','Mavie','Ottertail','Parkers Prairie','Perham','Perley','Pine Point','Rice','Roseau','Sabin','Staples','Stephen','Ulen','Wadena','Warren','Waukon','Westport','Williams'])
ndawnDF=exampleRequest.get_data()





Request successful


/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (9,15,19,21,23,25,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [121]:
ndawnDF

,Station Name,Latitude (deg),Longitude (deg),Elevation (ft),Year,Month,Day,Hour (CST),Avg Air Temp (Degrees F),Avg Air Temp Flag,...,Avg Wind Dir SD at 10 m Flag,Avg Wind Speed (mph),Avg Wind Speed Flag,Max Wind Speed (mph),Max Wind Speed Flag,Avg Wind Speed at 10 m (mph),Avg Wind Speed at 10 m Flag,Max Wind Speed at 10 m (mph),Max Wind Speed at 10 m Flag,Date
0,Ada,47.321100,-96.513900,910.0,2021.0,10.0,22.0,100.0,27.630,NaN,...,NaN,2.184,NaN,4.603,NaN,NaN,NaN,NaN,NaN,2021-10-22
1,Ada,47.321100,-96.513900,910.0,2021.0,10.0,22.0,200.0,27.968,NaN,...,NaN,0.809,NaN,3.235,NaN,NaN,NaN,NaN,NaN,2021-10-22
2,Ada,47.321100,-96.513900,910.0,2021.0,10.0,22.0,300.0,27.282,NaN,...,NaN,0.027,NaN,0.995,NaN,NaN,NaN,NaN,NaN,2021-10-22
3,Ada,47.321100,-96.513900,910.0,2021.0,10.0,22.0,400.0,26.298,NaN,...,NaN,0.018,NaN,0.995,NaN,NaN,NaN,NaN,NaN,2021-10-22
4,Ada,47.321100,-96.513900,910.0,2021.0,10.0,22.0,500.0,25.830,NaN,...,NaN,2.182,NaN,3.483,NaN,NaN,NaN,NaN,NaN,2021-10-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18812,Williams,48.858454,-94.980897,1093.0,2021.0,11.0,18.0,2100.0,23.124,NaN,...,NaN,5.730,NaN,9.330,NaN,NaN,NaN,NaN,NaN,2021-11-18
18813,Williams,48.858454,-94.980897,1093.0,2021.0,11.0,18.0,2200.0,22.726,NaN,...,NaN,4.928,NaN,7.712,NaN,NaN,NaN,NaN,NaN,2021-11-18
18814,Williams,48.858454,-94.980897,1093.0,2021.0,11.0,18.0,2300.0,20.764,NaN,...,NaN,3.680,NaN,5.598,NaN,NaN,NaN,NaN,NaN,2021-11-18
18815,Williams,48.858454,-94.980897,1093.0,2021.0,11.0,18.0,2400.0,19.157,NaN,...,NaN,3.781,NaN,6.718,NaN,NaN,NaN,NaN,NaN,2021-11-18


In [122]:
#Get the average temp for each station
avetemp=ndawnDF.groupby(['Station Name','Latitude (deg) ','Longitude (deg) ','Elevation (ft) ']).agg({'Avg Air Temp (Degrees F) ':['mean']})
avetemp.columns = avetemp.columns.droplevel(1)
avetemp=avetemp.reset_index()

In [124]:
#Export data to csv file
avetemp.to_csv('ndawnData-5.csv', index=False)

In [8]:
#Run TabletoTable to convert csv file
arcpy.conversion.TableToTable("ndawnData-5.csv", r"C:\Users\zhux0474\Desktop\lab3_4\Lab3&4\Lab4\MyProject\MyProject.gdb", "ndawn1", '', r'Station Name "Station Name" true true false 8000 Text 0 0,First,#,C:\Users\zhux0474\Desktop\lab3_4\Lab3&4\Lab4\ndawnData-5.csv,Station Name,0,8000;Latitude (deg) "Latitude (deg)" true true false 8 Double 0 0,First,#,C:\Users\zhux0474\Desktop\lab3_4\Lab3&4\Lab4\ndawnData-5.csv,Latitude (deg),-1,-1;Longitude (deg) "Longitude (deg)" true true false 8 Double 0 0,First,#,C:\Users\zhux0474\Desktop\lab3_4\Lab3&4\Lab4\ndawnData-5.csv,Longitude (deg),-1,-1;Elevation (ft) "Elevation (ft)" true true false 8 Double 0 0,First,#,C:\Users\zhux0474\Desktop\lab3_4\Lab3&4\Lab4\ndawnData-5.csv,Elevation (ft),-1,-1;Avg Air Temp (Degrees F) "Avg Air Temp (Degrees F)" true true false 8 Double 0 0,First,#,C:\Users\zhux0474\Desktop\lab3_4\Lab3&4\Lab4\ndawnData-5.csv,Avg Air Temp (Degrees F),-1,-1', '')

### IDW

In [4]:
#Run IDW to the ndawn data
arcpy.ga.IDW("ndawn_XYTableToPoint", "Avg_Air_Temp__Degrees_F_", "IDW_ndawn1", None, 0.0132, 2, "NBRTYPE=Standard S_MAJOR=1.20984183082956 S_MINOR=1.20984183082956 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", None)

id,value
0,a Layer object
1,


### kringing

In [6]:
#Run Kringing to the ndawn data
arcpy.sa.Kriging("ndawn_XYTableToPoint", "Avg_Air_Temp__Degrees_F_", "Spherical 0.013200 # # #", 0.0132, "VARIABLE 12", None); out_surface_raster.save(r"C:\Users\zhux0474\Desktop\lab3_4\Lab3&4\Lab4\MyProject\MyProject.gdb\Kriging_ndawn1")

### Radial Basis Functions

In [1]:
#Run RBF to the ndawn data
arcpy.ga.RadialBasisFunctions("ndawn_XYTableToPoint", "Avg_Air_Temp__Degrees_F_", "RBF_ndawn", None, 0.0132, "NBRTYPE=Standard S_MAJOR=1.20984183082956 S_MINOR=1.20984183082956 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR", "COMPLETELY_REGULARIZED_SPLINE", None)

id,value
0,a Layer object
1,
